# Bowls, A hand controllable gesture based synthesizer
Created using Modal synthesis of tibean drums 
https://www.music.mcgill.ca/~gary/618/week11/node12.html
- 586 Hz - Circulation and Sexuality 
- 492 Hz - Adrenals and Thyroid
- 330 Hz - Kidney 
- 198 Hz - Liver
- 352 Hz - Bladder
- 281.6 Hz - Small Intestine 
- 220 Hz - Lungs 
- 176 Hz - Colon 
- 330 Hz - Gallbladder 
- 117.3 Hz - Pancreas 
- 110 Hz - Stomach 
- 492 Hz - Spleen 

In [557]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal 
from aenum import Enum, skip
import time
import math
import os

# Friction Curves
source: https://www.engineeringtoolbox.com/friction-coefficients-d_778.html
https://engineeringlibrary.org/reference/coefficient-of-friction 

map touch sensor to circle aka sin() -1 to 1

$n$ = each node.

$V_{il}(n)$ and $V_{ir}(n)$ = incoming velocities at the excitation point arriveing from left to right

$V_{h}$ = $\sum_{n}$ $V_{il}(n)$ + $V_{ir}(n)$

In [558]:
class material(Enum):
    @skip
    class aluminum(Enum):
        static = 0.8
        dynamic = 0.98

# print(material.aluminum.static.value)


def frictionCurve(static: float, dynamic: float, initial_velocity: float, relative_velocity: float):
    return dynamic + ((static - dynamic) * initial_velocity) / (initial_velocity + relative_velocity)


# print(test)
# print(math.floor(1.1))
speed_in_air = 343000 / 44100

print(speed_in_air)
last_val = 0.5
current = 0.75

test = frictionCurve(material.aluminum.static.value, material.aluminum.dynamic.value, speed_in_air * 100, speed_in_air)
print(1800 / 44100)
testArray = np.zeros(2)
testArray[0]
testArray[1]

7.777777777777778
0.04081632653061224


0.0

# Bow Table
Ported from STK Bowtable.h
https://github.com/thestk/stk/blob/master/include/BowTable.h

In [559]:
class bowtable():
    def __init__(self, offset, slope, min_value, max_value):
        self.offset = offset
        self.slope = slope
        self.min_output = min_value
        self.max_output = max_value
    def process(self, input: float):
        sample = input + self.offset
        sample *= self.slope

        output = abs(sample + 0.75)
        output = output**-4

        if output < self.min_output:
            output = self.min_output
        if output > self.max_output:
            output = self.max_output

        return output



# Digital Wave Guide Filter
y(n) = 2Rcos(theta * y(n - 1)) - R^2y(n - 2) + x(n)
R = e^(-d/FS)
Theta = w / FS
w = 2pi f
FS = Sampling rate

In [560]:
# Truthfully, I don't beileve in healing frequencies. I just thought it would be funny to include
class Aura(Enum):
    Circulation: float = 585
    Adrenals: float = 492
    Thyroid: float = 492
    Kidney: float = 330
    Liver: float = 198
    Bladder : float = 352
    Small_Intestine: float = 281.6
    Lungs: float = 220
    Colon: float = 176
    Gallbladder: float = 330
    Pancreas: float = 117.3
    Stomach: float = 110
    Spleen: float = 492

# Fractional Delay Based off of Dafx Book Pg: 73-75
class fractional_delay:
    def __init__(self, M: float):
        # Get Delay length and 
        self.delay_length = M
        self.M = math.floor(M)
        self.frac = self.delay_length - self.M 

        # Read and Write Pointers
        self.read = 1
        self.write = 0

        # Init Delay line and set Delay Length to right Value
        self.delay = 0
        if self.delay_length > 2:
            self.delay = np.zeros(self.M)
            self.delay_length = self.M
        else:
            self.delay = np.zeros(2)
            self.delay_length = 2

        # Create pointer for Last Sample
        self.last_sample = self.delay_length - 1

    def process(self, input: float):
        # Get Output
        output = self.delay[self.last_sample] * self.frac + self.delay[self.read] * (1 - self.frac)
        self.delay[self.write] = input

        # Move Pointers
        self.read += 1
        self.write += 1
        self.last_sample += 1

        # Make sure pointers don't go out of bounds
        if self.write >= self.delay_length:
            self.write = 0
        if self.read >= self.delay_length:
            self.read = 0
        if self.last_sample >= self.delay_length:
            self.last_sample = 0

        return output

class wave_guide:
    def __init__(self, N: int):
        self.delay_line = np.zeros(N + 1)
        self.read: int = 1
        self.write: int = 0
        self.delay_length = N + 1

    def process(self, input: float):
        output = self.delay_line[self.read]
        self.delay_line[self.write] = input
        
        self.read += 1
        self.write += 1

        if self.write >= self.delay_length:
            self.write = 0
        if self.read >= self.delay_length:
            self.read = 0

        return output
    
    def get_output(self):
        return self.delay_line[-1]
    
    def get_delay_len(self):
        return len(self.delay_line)
    
    def get_read(self):
        return self.read
    
    def get_write(self):
        return self.write



class single_mode:
    # Arguments that don't need inits
    y1: float = 0
    y2: float = 0
    x1: float = 0
    x2: float = 0
    feedback: float = 0
    last_source: float = 0
    l1: float = 0
    l2: float = 0

    # Incoming Velocities Left and Right
    vil: float = 0
    vir: float = 0

    # Outgoing velocities Left and Right

    def __init__(self, samplingRate: float, frequency: float, dampening: float):
        # Init all these varibles on creation
        self.sampling_rate = samplingRate
        self.frequency = frequency
        self.dampening = dampening

        # Get the constants for the bp filter
        Bandwidth = 2
        R = 1 - (Bandwidth / 2)
        omega = math.cos(self.frequency)
        w = 2 * np.pi * self.frequency
        theta = w / self.sampling_rate
        theta = ((2 * R) / (1 + R*R)) * omega

        self.c2 = (R * R)
        self.c1 = 2 * R * theta
        self.c3 = R


        # Get Freq of waveguides and init
        # Delay Length = Samplerate / Frequency
        # k1: int = round(self.c2 / (freq_one + (self.c1 * freq_one**2)/self.frequency))
        # k2: int = round(self.c2 / (freq_two + (self.c1 * freq_two**2)/self.frequency))
        # k1: float = self.sampling_rate / (self.frequency)
        k1: float =  self.sampling_rate / (self.frequency)
        # k1: float = (self.frequency + beats) / 1000
        # k2: float = (self.frequency - beats) / 1000
        self.wave_guides = fractional_delay(k1)
        print("K1: ", k1)

    # There are faster ways but this works for now
    def __process_bandpass( self, input: float):
        # Process BP filter
        output = self.c1  * self.y1 - self.c2 * self.y2 - self.c3 * self.x2 + input
        self.x2 = self.x1
        self.x1 = input
        self.y2 = self.y1
        self.y1 = output
        return output
        
    def process(self, input: float):
        coupling_constant = 0.98
        self.last_source = 0
        
        
        # Create a simple matrix for the outputs
        output_matrix: float = 0
        bandpass_out: float = 0
        source: float = 0
        

        # Set Input
        source = input + self.feedback

        # Get bandpass Outs
        bandpass_out = self.__process_bandpass(source)

        # Get Output
        output_matrix = self.wave_guides.process(bandpass_out)

        #coupling_constant = frictionCurve(material.aluminum.static.value, material.aluminum.dynamic.value, input, output_matrix)

        # Set Feedback
        self.feedback = output_matrix * coupling_constant



        # Second half of waveguides
        #output_matrix[0,1] = self.l1 = self.wave_guides[0,1].process(bandpass_out)
        # # Bottom Right  
        # self.vir = output_matrix[1,1] = self.wave_guides[1,1].process(output_matrix[0,0])
        # # Bottom Left
        # self.vil = output_matrix[1,0] = self.wave_guides[1,0].process(output_matrix[0,1])
        
        # self.vil = output_matrix[0,1]
        # self.vir = output_matrix[1,1]
        # self.vol = output_matrix[1,0]
        # self.vor = output_matrix[0,0]

        #return [bandpass_out, output_matrix[0,1], output_matrix[1,1]]
        #return bandpass_out
        #mixed_out = output_matrix + output_matrix
        return output_matrix
    
test = single_mode(44100, 440, 38)



K1:  100.22727272727273


In [561]:
# Singing bowls
class singing_bowl_modes:
    def __init__(self, samplingRate: float, frequency: float, dampening: float):
        self.left_harmonics = np.array([single_mode(samplingRate, frequency + 0.9, dampening),
                              single_mode(samplingRate, frequency * 2 + 1, dampening),
                              single_mode(samplingRate, frequency * 3 + 1, dampening),
                              single_mode(samplingRate, frequency * 4 + 1, dampening),
                              ])
        self.right_harmonics = np.array([single_mode(samplingRate, frequency - 0.9, dampening),
                        single_mode(samplingRate, frequency * 2 - 1, dampening),
                        single_mode(samplingRate, frequency * 3 - 1, dampening),
                        single_mode(samplingRate, frequency * 4 - 1, dampening),
                        ])
        self.amp_values = np.array([0.9, 0.7, 1, 0.8])
        self.left_out: float = 0
    def process(self, input: float, debug= False):
        output = 0
        #right_out = 0
        

        # for i in range(0, 0):
        #     output += self.right_harmonics[i].process(input) * self.amp_values[i]
        #     output += self.left_harmonics[i].process(input) * self.amp_values[i]

        # Soundes worse with more harmonics
        output += self.right_harmonics[0].process(input)  * 0.5
        output += self.left_harmonics[0].process(input) * -0.5
        
        # for i in range(0, len(self.left_harmonics)):
        #     self.left_out += self.left_harmonics[i].process(output) * 0.09


        return output

# Test Runner Function

In [562]:
def internalTestRunner(outputFileName: str, fileSec: int, sampleRate: int, speed: int):
    print()
    print("### {} ###".format(outputFileName))
    sampleTime: any = sampleRate / speed
    impulse = np.zeros(sampleRate)

    valArray = np.rint(np.array([0, sampleTime, sampleTime * 2, sampleTime * 3]))
    numVal = 1
    impulse[0] = numVal
    # impulse[11025] = numVal
    # impulse[11025 * 2] = numVal
    # impulse[11025 * 3] = numVal
    
    # for i in range(0, speed):
    #     currentVal = np.rint(i * sampleTime)
    #     impulse[valArray[i]] = 1

    sine_wave: float = [1 - np.cos(2 * np.pi * x/(sampleRate)) for x in range(sampleRate)]
    fullImpulse = np.tile(impulse, fileSec)
    

    newFilter = singing_bowl_modes(sampleRate, Aura.Kidney.value, 38)
    #newFilter = single_mode(sampleRate, Aura.Kidney.value, 38, 1)
    outputSignal = np.zeros(len(fullImpulse))
    debug = True
    for i in range(0, len(fullImpulse)):
        if i != 0:
            debug = False            
        outputSignal[i] = newFilter.process(fullImpulse[i])

    
    outputLocation = "exports/" + outputFileName +  ".wav"
    sf.write(outputLocation, outputSignal, sampleRate)


### test1 ###
--- Opened File 0.00021314620971679688 seconds ---
--- File Info 0.0019483566284179688 seconds ---
--- Filter 0.010383129119873047 seconds ---


--- Sine Wave 1.1468422412872314 seconds ---
K1:  100.02268088001814
K1:  50.05675368898979
K1:  33.38380015140046
K1:  25.04258943781942
K1:  100.43270325666134
K1:  50.170648464163826
K1:  33.434420015163
K1:  25.071063104036384
--- Init 1.1471490859985352 seconds ---
--- Process 3.402174234390259 seconds ---
--- Write File 3.4060983657836914 seconds ---
Input:  0.6242950769699741 Output:  0.0
Input:  0.6242950319111448 Output:  0.6242950769699741
Input:  0.624294896734695 Output:  0.6242950319111448
Input:  0.6242946714407434 Output:  0.624294896734695
Input:  0.6242943560294831 Output:  0.6242946714407434
Input:  0.6242939505011886 Output:  0.6242943560294831
Input:  0.6242934548562106 Output:  0.6242939505011886
Input:  0.6242928690949787 Output:  0.6242934548562106
Input:  0.6242921932180007 Output:  0.6242928690949787
Input:  0.6242914272258608 Output:  0.6242921932180007
Input:  0.6242905711192229 Output:  0.6242914272258608


In [563]:

bow_offset: float = 3
bow_slope: float = 0.125
bow_min: float = 0
bow_max: float = 4

def testRunner(inputFile, outputFileName):
    print()
    print("### {} ###".format(outputFileName))
    start_time = time.time()    
    inputLocation = "imports/" + inputFile
    with sf.SoundFile(inputLocation) as input_file:
        print("--- Opened File %s seconds ---" % (time.time() - start_time))
        y =  np.asarray(input_file.read()) # Read input File
        Fs = input_file.samplerate # Get sample rate
        int_sample = int(Fs)
        fileLength = len(y) # Get length of input file
        fileSec = fileLength / Fs # Get length of input file in seconds
        impulse = np.zeros(fileLength)
        outputSignal = np.zeros(fileLength)
        #impulse[1] = 1
        freq = 440

        print("--- File Info %s seconds ---" % (time.time() - start_time))

        sos = signal.butter(2, freq, 'low', fs=Fs, output='sos')
        filtered = signal.sosfilt(sos, y)
        print("--- Filter %s seconds ---" % (time.time() - start_time))
        # sos2 = signal.butter(2, freq, 'high', fs=Fs, output='sos')
        # filtered2 = signal.sosfilt(sos2, filtered)

        bow_table = bowtable(bow_offset, bow_slope, bow_min, bow_max)
        sine_wave: float = [bow_table.process(1 - (np.cos(2 * np.pi * x * 4 / int(Fs)))) for x in range(int(Fs * fileSec))]
        y = np.add(0, filtered)
        print("--- Sine Wave %s seconds ---" % (time.time() - start_time))
        newFilter = singing_bowl_modes(Fs, 440, 38)
        print("--- Init %s seconds ---" % (time.time() - start_time))
        for i in range(0, len(y)):
            outputSignal[i] = newFilter.process(filtered[i])
        print("--- Process %s seconds ---" % (time.time() - start_time))

        #outputSignal = [] # Output signal
        # for i in range(0, len(y)):
        #     outputSignal.append((newout[i]))

        outputLocation = "exports/" + outputFileName +  ".wav"
        sf.write(outputLocation, outputSignal, Fs)
        print("--- Write File %s seconds ---" % (time.time() - start_time))
    return "Successfully Wrote to File"



def delay_test(outputFileName: str, fileSec: int, sampleRate: int):
    delay_length = round(sampleRate * (fileSec / 2))
    delay_time = 0.75
    delay_line = fractional_delay(delay_time)
    #print("Ideal Length: ", delay_length, "Real Length: ", delay_line.get_delay_len())
    impulse = np.zeros(sampleRate * fileSec)
    outputSignal = np.zeros(sampleRate)
    impulse[0] = 1

    bow = bowtable(bow_offset, bow_slope, bow_min, bow_max)
    sine_wave: float = [bow.process(1 - np.cos(2 * np.pi * 4 * x/(sampleRate))) for x in range(sampleRate)]

    # for i in range(0, len(sine_wave)):
    #     impulse[i] = sine_wave[i]

    for i in range(0, len(sine_wave)):
        outputSignal[i] = delay_line.process(sine_wave[i])
        if i < delay_time + 10:
            print("Input: ", sine_wave[i], "Output: ", outputSignal[i])
        #print(i, "Input: ", impulse[i], "Output: ", outputSignal[i], "read: ", delay_line.get_read())

    # outputLocation = "exports/" + outputFileName +  ".wav"
    # sf.write(outputLocation, outputSignal, sampleRate)

    fig, ax1 = plt.subplots(1, 1, layout='constrained')
    ax1.plot(sine_wave)
    ax1.plot(outputSignal)

testRunner("whitenoise.wav", "test1")
# internalTestRunner("test2", 5, 44100, 4)
# testRunner("whitenoise_strike.wav", "test3")
delay_test("delay_test", 1, 44100)
